In [ ]:
import numpy as np
import random
import sympy as sp
import numpy as np
import numpy.linalg as la

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
wx = sp.symbols('w_1:3')
sx = sp.symbols('s_1:3')
#px = ()
px = sp.symbols('p_1:3')

def getA(s,w,p):
    if not isinstance(w, tuple):
        w = (w,)
    if not isinstance(s, tuple):
        s = (s,)
    if not isinstance(p, tuple):
        p = (p,)
    A = sp.zeros(len(2*w+p), len(2*w)+len(p))
    for i,(v,r) in enumerate(zip(w,s)):
        XX = sp.Matrix([[r, v],[-v,r]])
        ix = 2*i
        A[ix, ix] = r
        A[ix+1, ix+1] = r
        A[ix, ix+1] = v
        A[ix+1, ix] = -v
    for i,r in enumerate(p):
        A[len(2*w)+i, len(2*w)+i] = r
    return A
#sp.pprint(getA(wx,sx,px))


def to_sympy_vector(syms):
    return sp.Matrix(syms)
#sp.pprint(to_sympy_vector(wx+sx))

def get_value_by_symbol(values, all_syms, syms):
    res = []
    for sy in syms:
        idx = list(all_syms).index(sy)
        res.append(values[idx])
    return np.matrix(np.array(res).squeeze())

def get_accessor(all_syms_vector, syms):
    return sp.lambdify(all_syms_vector, syms)

assert(np.all(get_value_by_symbol(np.arange(len(sx+wx)), sx+wx, sx) == np.arange(len(sx))))

In [ ]:
### Define System model
num_cplx_poles = 1
num_real_poles = 0

# define symbols
#w = sp.symbols('w_1:2') # creates tuple (w_1) 
#s = sp.symbols('s_1:2') # creates tuple (s_1) 
#b = sp.symbols('b_1:3') # creates tuple (b_1, b_2)
#d = sp.symbols('d_1:2') # creates tuple (d_1) 
#x = sp.symbols('x_1:3') # creates tuple (x_1, x_2) 
#c = sp.symbols('c_1:3') #creates tuple (c_1, c_2)
    
if num_cplx_poles == 0:
    w = ()
    s = ()
else:
    w = sp.symbols('w_1:{}'.format(num_cplx_poles+1)) # creates tuple (w_1, w_2)
    s = sp.symbols('s_1:{}'.format(num_cplx_poles+1)) # creates tuple (s_1, s_2)

if num_real_poles == 0:
    p = ()
else:
    p = sp.symbols('s_1:{}'.format(num_real_poles+1)) # creates tuple (s_1, s_2)

b = sp.symbols('b_1:{}'.format(2*num_cplx_poles+num_real_poles+1)) # creates tuple (b_1, b_2, b_3, b_4)
d = sp.symbols('d_1:2') # creates tuple (d_1)
x = sp.symbols('x_1:{}'.format(2*num_cplx_poles+num_real_poles+1)) # creates tuple (x_1, x_2, x_3, x_4)
c = sp.symbols('c_1:{}'.format(2*num_cplx_poles+num_real_poles+1)) # creates tuple (c_1, c_2, c_3, c_4)

u = sp.symbols('u_1:2')

# define structures / matrizes / vectors of symbols
X = to_sympy_vector(x)
U = to_sympy_vector(u)

A = getA(s, w, p)
B = to_sympy_vector(b)
C = to_sympy_vector(c).T
D = to_sympy_vector(d)

AA = sp.lambdify(s+w+p, A)
BB = sp.lambdify(b, B)
CC = sp.lambdify(c, C)
DD = sp.lambdify(d, D)

z = x+s+w+p+b+c+d
Z = to_sympy_vector(z)

zu = z+u

fx = A*X+B*U

f = sp.Matrix([0 for _ in range(len(z)-len(x))])
f = fx.col_join(f)

print "f="; sp.pprint(f)
ff = sp.lambdify(zu, f)

print "F="; sp.pprint(f.jacobian(z))
FF = sp.lambdify(zu, f.jacobian(z))

h = C*X+D*U
print "h="; sp.pprint(h)
hh = sp.lambdify(zu, h)

print "H="; sp.pprint(h.jacobian(z))
HH = sp.lambdify(zu, h.jacobian(z))


In [ ]:
# Unroll for np.matrix instead of np.array
def _c(M):
    sh = M.shape
    if sh[0] == 1:
        return M.tolist()[0]
    elif sh[1] == 1:
        return M.T.tolist()[0]
    else:
        print "WARNING! Unrolling can go horribly wrong with matrizes!"
        return M.ravel().tolist[0]
        
zu_array = np.random.rand(len(zu))
#print ff(*zu_array)

zu_matrix = np.matrix(zu_array)
#print ff(*_c(zu_matrix))

assert(np.all(ff(*zu_array) == ff(*_c(zu_matrix))))

In [ ]:
rho = 0.1

#R = np.eye(len(z))
R = 1
print "R:", R

q = np.array([])
for sym in z:
    if sym in X.T.row_join(C):
        q = np.hstack((q, 0))
    else:
        q = np.hstack((q, 1))

Q = rho * np.matrix(np.diag(q))
print "Q:\n", Q

In [ ]:
import control as con

Af = np.matrix([[  -2, 4],
                [  -4, -2]])

Bf = np.matrix([[ 1],
                [ 1]])

Cf = np.matrix([[ 1/0.3, 0.]])

Df = np.matrix([[ 0.]])

ss2 = con.matlab.ss(Af, Bf, Cf, Df)
T2, yout2 = con.matlab.step(ss2)
plt.plot(yout2, T2)
plt.show()
print con.dcgain(ss2)
print ss2.pole()

In [ ]:
### Testsystem ####
def calc_TestSystem(x, u):
    global Af, Bf, Cf, Df

    x_dot = Af.dot(x) + Bf.dot(u)
    y = Cf.dot(x) + Df.dot(u)
    
    #print x_dot
    return y, x_dot

In [ ]:
# return real part of a vector
def real_vector(vector):
    return map(lambda x: x.real, vector)

# return imaginary part of a vector
def imag_vector(vector):
    return map(lambda x: x.imag, vector)

In [ ]:
%%time
# initial
random.seed(42) # Set the random number generator to a fixed sequence.

# A
#s_k0 = np.matrix([-2])#, -2.1]) # real Teil
s_k0 = np.matrix(-1*(1.0+np.random.rand(len(s))))#, -2.1]) # real Teil

#w_k0 = np.matrix([-2.2])#, -2.3])  # (+/-) imag Teil
w_k0 = np.matrix((1.0+np.random.rand(len(w))))#, -2.1]) # real Teil

p_k0 = np.matrix((-1.0+np.random.rand(len(p)))) # rein real Teil


#b_k0 = np.matrix([3.1, 3.2])#, 3.3, 3.4])
b_k0 = np.matrix((np.random.rand(len(b))))#, -2.1]) # real Teil

#c_k0 = np.matrix([4.1, 4.2])#, 4.3, 4.4])
c_k0 = np.matrix((np.random.rand(len(c))))#, -2.1]) # real Teil

d_k0 = np.matrix(np.zeros(len(d)))

P_k0 = np.eye(len(z))

x_k0 = np.matrix(np.zeros(len(x))).T#, 0, 0]).T

u_k0 = np.matrix([0.1])

T = 0.001


# First values
x_k = x_k0
x_t = np.matrix([0, 0]).T
u_k = u_k0
z_k = np.hstack((x_k0.T, s_k0, w_k0, p_k0, b_k0, c_k0, d_k0)).T
#print "z_k:", z_k

P_k = P_k0

# naive Logging 
poles = []
y = []
y_e = []

u = []

# Create accessors
x_from_z = get_accessor(Z, x)
s_from_z = get_accessor(Z, s)
w_from_z = get_accessor(Z, w)
p_from_z = get_accessor(Z, p)

swp_from_z = get_accessor(Z, s+w+p)

b_from_z = get_accessor(Z, b)
c_from_z = get_accessor(Z, c)
d_from_z = get_accessor(Z, d)

# Loop start
#step_time = np.random.randint(100)*T + 0.1
step_time = 0.5
for cnt, t in enumerate(np.arange(T, 30, T)):

    # Every two seconds new step
    if np.isclose(t%step_time, 0, T):
        u_k = u_k0 + random.uniform(-1, 1)
        #step_time = np.random.randint(100)*T + 0.1
        print "step:", t, "u:", u_k, "trace(P)", np.trace(P_k)
        s_k = np.matrix(s_from_z(*_c(z_k))).T
        w_k = np.matrix(w_from_z(*_c(z_k))).T
        p_k = np.matrix(p_from_z(*_c(z_k))).T
        poles.append(np.array([complex(si,wi) for si,wi in zip(s_k, w_k)] +
                              [complex(si,-wi) for si,wi in zip(s_k, w_k)] +
                              [pi for pi in p_k]))
        
    y_t, x_dot = calc_TestSystem(x_t, u_k)
        
    y.append(y_t[0,0])
    u.append(u_k[0,0])
    
    ##### Reconstruction from z
    x_k = np.matrix(x_from_z(*_c(z_k))).T
    
    swp_k = np.matrix(swp_from_z(*_c(z_k)))

    b_k = np.matrix(b_from_z(*_c(z_k)))
    c_k = np.matrix(c_from_z(*_c(z_k)))
    d_k = np.matrix(d_from_z(*_c(z_k)))

    
    ## System
    A_k = AA(*_c(swp_k))
    B_k = BB(*_c(b_k))
    C_k = CC(*_c(c_k))
    D_k = DD(*_c(d_k))


    ##### Evaluation
    # State space
    dx_k = A_k.squeeze().dot(x_k) + B_k.dot(u_k)
    y_k = C_k.dot(x_k) + D_k.dot(u_k)
    y_e.append(y_k[0,0])

    # Concate values for lambdafied Jacobians
    zu_k = np.hstack((z_k.T, u_k))
    #print "zu_k:\n", zu_k
    
    h_k = np.matrix(hh(*_c(zu_k))) # h = y_k (predicted)
    #print "h_k:\n", h_k
    H_k = np.matrix(HH(*_c(zu_k))) # h.Jacobian
    #print "H_k:\n", H_k

    
    f_k = np.matrix(ff(*_c(zu_k))) # f
    #print "f_k:\n", f_k
    F_k = np.matrix(FF(*_c(zu_k))) #f.Jacobian
    #print "F_k:\n", F_k
    
    ##### Kalman Filter

    # Prediction
    
    K_k = P_k.dot(H_k.T).dot(la.inv(H_k.dot(P_k).dot(H_k.T) + R))
    #print "K_k:\n", K_k

    Ps_k = (np.eye(len(z)) - K_k.dot(H_k)).dot(P_k) # P*_k
    #print "Ps_k:\n", Ps_k

    # Correction
    P_k1 = Ps_k + T*(F_k.dot(Ps_k) + Ps_k.dot(F_k.T) + Q)
    #print "P_k1:\n", P_k1

    z_k1 = (z_k + T*f_k + K_k.dot((y_t - h_k)))
    #print "z_k1:", z_k1

    # State Propagation
    x_t = x_t + x_dot*T
    #print "x_k:\n",x_k
    
    z_k = np.matrix(z_k1)
    #print "z_k:\n", z_k
    
    P_k = np.matrix(P_k1).astype(np.float)
    #print "P_k:\n", P_k


# another approach to plot
real_part = real_vector(poles)
imag_part = imag_vector(poles)

# Display a window with a plot of real, imag
plt.plot(real_part, imag_part, 'b-')
plt.plot(real_vector(poles), imag_vector(poles), 'gx')
plt.plot(real_part[0], imag_part[0], 'r.')
plt.plot(real_part[-1], imag_part[-1], 'ro')

plt.show

print

In [ ]:
plt.plot(y, 'b')
plt.plot(u, 'r')
plt.plot(y_e, 'y-')

plt.show

In [ ]:
ss = con.matlab.ss(A_k, b_k.T, c_k, d_k)

#print A_k[:,:,0,0]
#print A_k.squeeze()
#print b_k
#print c_k
#print d_k

#print poles[-1]
print ss

In [ ]:
print con.dcgain(ss2)
print ss2.pole()
print con.dcgain(ss)
print ss.pole()

In [ ]:
T3, yout3 = con.matlab.step(ss2)
plt.plot(yout3, T3)
#plt.show()

T4, yout4 = con.matlab.step(ss)
plt.plot(yout4, T4)

plt.show()